In [1]:
import os
import pandas as pd


In [2]:
# Función para leer todos los archivos .xlsx en el directorio
def leer_archivos_xlsx(directorio):
    # Lista para almacenar los DataFrames
    dfs = []
    
    # Iteramos sobre todos los archivos en el directorio
    for archivo in os.listdir(directorio):
        if archivo.endswith('.xlsx'):
            # Construimos la ruta completa del archivo
            ruta_archivo = os.path.join(directorio, archivo)
            
            # Leemos el archivo Excel y lo agregamos a la lista
            df = pd.read_excel(ruta_archivo)
            dfs.append(df)
    
    return dfs


In [3]:
# Función para unir los DataFrames
def unir_dataframes(dfs):
    # Inicializamos un DataFrame vacío
    df_final = pd.DataFrame()
    
    # Iteramos sobre cada DataFrame en la lista
    for df in dfs:
        # Unimos el DataFrame actual con el final, usando las columnas comunes
        df_final = pd.concat([df_final, df], axis=0, ignore_index=True, join='outer')
    
    return df_final


In [4]:
# Ruta de entrada (donde están los archivos .xlsx)
directorio_entrada = r".\Proyecto Henry 2"  # Cambia esta ruta a la de tu carpeta de archivos .xlsx
directorio_salida = r".\Proyecto Henry 2\Dataframes"  # Cambia esta ruta a tu carpeta de salida


# Verificamos si el directorio de salida existe; si no, lo creamos
if not os.path.exists(directorio_salida):
    os.makedirs(directorio_salida)



In [ ]:
# Leemos los archivos y combinamos los DataFrames
archivos_dfs = leer_archivos_xlsx("C:\\Proyecto Henry 2")
df_combinado = unir_dataframes(r"C:\Proyecto Henry 2\Dataframes")



In [ ]:

¡Claro! A continuación, te muestro cómo quedaría la celda 6 completa con la lógica para que el archivo combinado mantenga el mismo nombre que el archivo original, añadiendo el sufijo "_combinado" antes de la extensión .xlsx.

Celda 6 modificada:
python
Copiar
# Obtener el nombre del primer archivo en el directorio de entrada
primer_archivo = os.listdir(directorio_entrada)[0]  # Toma el primer archivo encontrado
nombre_base = os.path.splitext(primer_archivo)[0]  # Extrae el nombre sin la extensión

# Crear el nuevo nombre del archivo añadiendo "_combinado"
nombre_combinado = f"{nombre_base}_combinado.xlsx"

# Ruta completa para guardar el archivo combinado en el directorio de salida
archivo_salida = os.path.join(directorio_salida, nombre_combinado)

# Guardamos el DataFrame combinado en el archivo con el nombre modificado
df_combinado.to_excel(archivo_salida, index=False)

# Verificamos la ruta del archivo guardado
archivo_salidaÑ


In [1]:
import pandas as pd
import os
import warnings

# Suprimir advertencias de conversión de fechas
warnings.simplefilter(action='ignore', category=UserWarning)

# Definir las rutas de los archivos
input_dir = r"C:\Users\Joseph´S - PC\OneDrive\Escritorio\Proyecto Henry 2"
output_dir = r"C:\Users\Joseph´S - PC\OneDrive\Escritorio\Proyecto Henry 2\Procesados"

# Obtener lista de archivos .xlsx en el directorio
xlsx_files = [f for f in os.listdir(input_dir) if f.endswith('.xlsx')]

# Asegurarse de que el directorio de salida existe
os.makedirs(output_dir, exist_ok=True)

print()



In [2]:
for file in xlsx_files:
    # Leer el archivo .xlsx en un diccionario de DataFrames (cada hoja se convierte en un DataFrame)
    file_path = os.path.join(input_dir, file)
    df_sheets = pd.read_excel(file_path, sheet_name=None)  # Carga todas las hojas
    
    # Crear un diccionario para almacenar las hojas procesadas
    processed_sheets = {}
    
    # Paso 2: Convertir columnas numéricas y de fecha para cada hoja
    for sheet_name, df in df_sheets.items():
        # Procesar las columnas del DataFrame
        for column in df.columns:
            try:
                # Intentar convertir a numérico
                df[column] = pd.to_numeric(df[column])
            except ValueError:
                pass  # Si no es posible convertir a numérico, no hacer nada

            try:
                # Intentar convertir a fecha
                df[column] = pd.to_datetime(df[column])
            except ValueError:
                pass  # Si no es posible convertir a fecha, no hacer nada
        
        # Almacenar la hoja procesada en el diccionario
        processed_sheets[sheet_name] = df
    
    # Paso 3: Guardar todas las hojas procesadas en un nuevo archivo .xlsx
    processed_file_path = os.path.join(output_dir, f"procesado_{file}")
    with pd.ExcelWriter(processed_file_path, engine='xlsxwriter') as writer:
        for sheet_name, processed_df in processed_sheets.items():
            processed_df.to_excel(writer, sheet_name=sheet_name, index=False)


In [3]:
import pandas as pd
import os
import warnings

# Suprimir advertencias de conversión de fechas
warnings.simplefilter(action='ignore', category=UserWarning)

# Definir las rutas de los archivos
input_dir = r"C:\Users\Joseph´S - PC\OneDrive\Escritorio\Proyecto Henry 2"

# Obtener lista de archivos .xlsx en el directorio y omitir los que contienen '(1)' en su nombre
xlsx_files = [f for f in os.listdir(input_dir) if f.endswith('.xlsx') and "(1)" not in f]

# Procesar cada archivo
for file in xlsx_files:
    file_path = os.path.join(input_dir, file)
    try:
        # Leer todas las hojas del archivo
        excel_data = pd.ExcelFile(file_path)
        print(f"Archivo: {file}")
        
    except Exception as e:
        print(f"Error al procesar el archivo {file}: {e}")




Archivo: Internet.xlsx
Archivo: mapa_conectividad.xlsx
Archivo: Portabilidad.xlsx
Archivo: servicios_postales.xlsx
Archivo: telefonia_fija.xlsx
Archivo: Telefonia_movil.xlsx
Archivo: Television.xlsx
